In [ ]:
# This file imports an EXCEL model (here template example from BW25 sharepoint folder) and matches it with the background database in the project. For this repository we are currently working with ecoinvent310clca

# After successfully importing a database to a Brightway project, the database is stored in the project and doesn't have to be imported every time we open the repository. Unless we make changes to the background database of course

# EXCEL Model importer

#### 1. Import packages

In [4]:
# basic imports from brightway
import bw2analyzer as ba
import bw2calc as bc
import bw2data as bd
from bw2data import databases
import bw2io as bi
from bw2io import ExcelImporter
from bw2io.importers import SingleOutputEcospold2Importer
import bw2analyzer as bwa
from bw2data import methods
import argparse
import bw2data as bd

# other relevant packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

12:31:27+0200 [warning  ] Can't import `SimaProBlockCSVImporter` - please install `bw2io` with `pip install bw2io[multifunctional]` or install `multifunctional` and `bw_simapro_csv` manually.


#### 2. Set project and see databases

In [5]:
# define a project where we install the databases and work in this script
bd.projects.set_current('brightway25')

In [6]:
bd.databases

Databases dictionary with 4 object(s):
	biosphere3
	bw25_db
	ecoinvent-3.10-biosphere
	ecoinvent310clca

In [7]:
ei_clca = bd.Database('ecoinvent310clca')
ei_bio = bd.Database('ecoinvent-3.10-biosphere')
db_template = bd.Database('bw25_db')
el_bio3 = bd.Database('biosphere3')

#### 3. Import the EXCEL file containing the model 

In [8]:
excel_db = bi.ExcelImporter("C:\\Users\\TimWeber\\OneDrive - 2.-0 LCA Consultants ApS\\Intranet - Brightway2\\excel_model_example\\bw25_model_example.xlsx")

Extracted 1 worksheets in 0.05 seconds


#### 4. Importing means matching with background databases and writing the database

In [9]:
excel_db.apply_strategies()

Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 0.65 seconds


In [10]:
excel_db.match_database(fields=['name','location'])

Applying strategy: link_iterable_by_fields


In [11]:
excel_db.match_database(db_name='ecoinvent310clca',
                        fields=['name','location','unit'])

Applying strategy: link_iterable_by_fields


In [12]:
excel_db.match_database(db_name='ecoinvent-3.10-biosphere',
                        fields=['name','categories','unit'])

Applying strategy: link_iterable_by_fields


In [13]:
excel_db.match_database(db_name='biosphere3',
                        fields=['name','categories','unit'])

Applying strategy: link_iterable_by_fields


In [14]:
# check foreground imports for unlinked processes
pd.DataFrame(excel_db.unlinked)

""


In [15]:
excel_db.write_database()

12:31:32+0200

 [warning  ] Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 1/1 [00:00<00:00, 313.64it/s]


12:31:33+0200 [info     ] Vacuuming database            
Created database: bw25_db


In [16]:
# if there are unlinked exchanges, this helps localizing them in order to fix them
list(excel_db.unlinked)


[]

#### 5. Store data in a dataframe so it can be analysed in Brightway 2.5

In [17]:
data_db_template = [activity.as_dict() for activity in db_template]


In [18]:
df_db_template = pd.DataFrame(data_db_template)


In [19]:
from IPython.display import display
display(df_db_template)

,comment,location,production amount,reference product,unit,name,worksheet name,database,code,type,id
0,This dataset represents production of porcelai...,RER,1,porcelain ceramics production,kilogram,porcelain ceramics production,LCI,bw25_db,a1b7153986d8f8387752b14278a0b4c2,processwithreferenceproduct,218678817873801216


In [20]:
# Get all activities and products
activities = set(db_template)  # All activities in the database

# Get all products linked via production exchanges
products = set([
    exc.input for act in db_template
    for exc in act.exchanges() if exc.input == act.key
])

# Identify mismatches
extra_activities = activities - products
extra_products = products - activities

print("Extra activities (no matching products):", extra_activities)
print("Extra products (no matching activities):", extra_products)

Extra activities (no matching products): set()
Extra products (no matching activities): set()


In [21]:
# what also works to show all activities in a database is the following
for act in db_template:
    print(act)

'porcelain ceramics production' (kilogram, RER, None)
